# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [9]:
#Your code here
#we need to import our libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
# we need to load our data set
df = pd.read_csv('homepage_actions.csv')

In [10]:
# let us see our data quickly
print(df.head())
print(df.info())
print(df['group'].value_counts())
print(df['action'].value_counts())

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
control       4264
experiment    3924
Name: group, dtype: int64
view     6328
click    1860
Name: action, dtype: int64


In [12]:
# Check for viewers who also clicked
views = df[df['action'] == 'view']
clicks = df[df['action'] == 'click']

# Check if anyone clicked without viewing
clickers = clicks['id'].unique()
viewers = views['id'].unique()
clickers_without_view = [id for id in clickers if id not in viewers]

print(f"Number of viewers: {len(viewers)}")
print(f"Number of clickers: {len(clickers)}")
print(f"Number of clickers who didn't view: {len(clickers_without_view)}")

# Check for overlap between control and experiment groups
control_ids = df[df['group'] == 'control']['id'].unique()
experiment_ids = df[df['group'] == 'experiment']['id'].unique()
overlap = set(control_ids) & set(experiment_ids)

print(f"Number of users in both groups: {len(overlap)}")

Number of viewers: 6328
Number of clickers: 1860
Number of clickers who didn't view: 0
Number of users in both groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [13]:
#Your code here
# Create a dataframe with click-through information for each user
user_actions = df.groupby(['id', 'group'])['action'].apply(list).reset_index()
user_actions['clicked'] = user_actions['action'].apply(lambda x: 1 if 'click' in x else 0)

# Calculate click-through rates
control_ctr = user_actions[user_actions['group'] == 'control']['clicked'].mean()
experiment_ctr = user_actions[user_actions['group'] == 'experiment']['clicked'].mean()

print(f"Control CTR: {control_ctr:.4f}")
print(f"Experiment CTR: {experiment_ctr:.4f}")

# Perform chi-square test
control_clicks = user_actions[user_actions['group'] == 'control']['clicked'].sum()
experiment_clicks = user_actions[user_actions['group'] == 'experiment']['clicked'].sum()
control_views = len(user_actions[user_actions['group'] == 'control'])
experiment_views = len(user_actions[user_actions['group'] == 'experiment'])

contingency_table = np.array([
    [control_clicks, control_views - control_clicks],
    [experiment_clicks, experiment_views - experiment_clicks]
])

chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table, correction=False)
print(f"Chi-square test p-value: {p_value:.6f}")

Control CTR: 0.2797
Experiment CTR: 0.3097
Chi-square test p-value: 0.008830


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [14]:
#Your code here
p_control = control_ctr
expected_clicks_experiment = experiment_views * p_control
print(f"Expected clicks for experiment group: {expected_clicks_experiment:.2f}")

Expected clicks for experiment group: 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [16]:
#Your code here
actual_clicks_experiment = experiment_clicks
variance = experiment_views * p_control * (1 - p_control)
std_dev = np.sqrt(variance)
z_score = (actual_clicks_experiment - expected_clicks_experiment) / std_dev
print(f"Z-score: {z_score:.4f}")

Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [17]:
#Your code here
p_value_z = 1 - stats.norm.cdf(z_score)  # One-tailed test
print(f"P-value from z-test: {p_value_z:.6f}")

P-value from z-test: 0.000125


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
1. Z-scores match perfectly - Your manual calculation of 3.6625 is statistically sound and uses the correct binomial variance formula (n × p × (1-p)).

2. P-values are consistent when corrected - Your one-tailed p-value of 0.000125 becomes 0.000250 when properly converted to two-tailed, which is very close to what we'd expect.

3. Chi-square test provides additional confirmation - The p-value of 0.008830 is slightly different due to different test assumptions but reaches the same conclusion.

All methods agree on significance - Every approach confirms the difference is statistically significant (p < 0.05).

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.